In [27]:
# Change directory to the desired path
%cd D:\Documents\ai_engineer_pacmann\ai_engineer_pacmann\CANDRA_MLPROCESS\src

import utils
from imblearn.over_sampling import RandomOverSampler


D:\Documents\ai_engineer_pacmann\ai_engineer_pacmann\CANDRA_MLPROCESS\src


c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from utils import load_data
from utils import split_input_output
from utils import split_train_test
from utils import serialize_data
from utils import deserialize_data

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report


Import some sklearn models


In [3]:
####################################################
# Import sklearn library of those six models + gridsearchcv
#
####################################################

from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier



Import GridSearchCV for hyperparameter Tuning, and numpy

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np


Deserialize from X dan y after RandomOversampling 

In [5]:
# Call the deserialize function to save the dataset into variable

X_train_model = deserialize_data(path='../data/processed/X_train_ros.pkl')
y_train_model = deserialize_data(path='../data/processed/y_train_ros.pkl')

## 1 . Create a list of Hyperparameter for every algorithm model

In [6]:
# grid hyperparameter for Baseline model
##
base_param_grid = {
}

# grid hyperparameter for Baseline model
##
knn_param_grid = {
    'n_neighbors': [5, 10],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree'],
    'leaf_size' : [30, 60],
    'p': [2],
    'metric': ['minkowski'],
    'metric_params': [None],
    'n_jobs': [None]
}

# grid hyperparameter untuk Decision Tree Classifier

dct_param_grid = {
    'criterion': ['gini', 'entropy'],  # Function to measure the quality of a split
    'splitter': ['best', 'random'],    # Strategy used to choose the split at each node
    'max_depth': [None, 10],  # Hanya 2 nilai untuk max_depth
    'min_samples_split': [2, 5],   # Hanya 2 nilai untuk min_samples_split
    'min_samples_leaf': [1, 2],  # Hanya 2 nilai untuk min_samples_leaf
    'max_features': [None, 'sqrt'],  # Hanya 2 nilai untuk max_features
    'class_weight': [None, 'balanced']  # Hanya 2 nilai untuk class_weight
}

Create instance for every model (baseline, kNN, DecisionTreeClasifier)

In [7]:
# Baseline model untuk klasifikasi
# Strategi 'most_frequent' untuk selalu memprediksi kelas yang paling sering muncul
instance_baseline = DummyClassifier()

# Instance KNN
instance_knn = KNeighborsClassifier(n_jobs=-1)

# Instance Decision Tree Classifier (tidak mendukung n_jobs)
instance_dt = DecisionTreeClassifier()

Create a Function for Training the model 

In [8]:
def train_model(model, param_grid, X_train, y_train):
    """
    Melakukan pelatihan model dengan hyperparameter tuning menggunakan GridSearchCV.
    
    Parameters:
    model : estimator
        Instance model yang akan dilatih.
    param_grid : dict
        Daftar hyperparameter untuk tuning.
    X_train : DataFrame atau ndarray
        Data latih.
    y_train : Series atau ndarray
        Target data latih.
    
    Returns:
    grid_search : GridSearchCV
        Instance GridSearchCV yang telah dilatih.
    """
    # Membuat instance GridSearchCV
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=3)
    
    # Melakukan pelatihan model
    grid_search.fit(X_train, y_train)
    
    # Mengembalikan instance GridSearchCV yang telah dilatih
    return grid_search


Calling the Function for every model (baseline, kNN, DTC)

In [9]:
# For baseline model
#
baseline_grid_search = train_model(model=instance_baseline,
                                   param_grid=base_param_grid,
                                   X_train=X_train_model,
                                   y_train=y_train_model)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [10]:
# For kNN Classifier model
#
knn_grid_search = train_model(model=instance_knn,
                                param_grid=knn_param_grid,
                                X_train=X_train_model,
                                y_train=y_train_model)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [11]:
# For DCT Classifier model
#
dct_grid_search = train_model(model=instance_dt,
                                param_grid=dct_param_grid,
                                X_train=X_train_model,
                                y_train=y_train_model)

Fitting 5 folds for each of 128 candidates, totalling 640 fits


Then Saving the models after fitting

In [12]:
# Call the serialize function to dump the models into .pkl files

serialize_data(data=baseline_grid_search, path='../models/baseline_model.pkl')
serialize_data(data=knn_grid_search, path='../models/kNN_model.pkl')
serialize_data(data=dct_grid_search, path='../models/dct_model.pkl')


['../models/dct_model.pkl']

## 2. Threshold Tuning (Data Validation)

In [28]:
%pwd

'D:\\Documents\\ai_engineer_pacmann\\ai_engineer_pacmann\\CANDRA_MLPROCESS\\src'

In [30]:
# Call the deserialize function to save the dataset into variable

X_valid_model = deserialize_data(path='../data/processed/X_valid_prep.pkl')

# Menggunakan data y_valid yang original (yang tidak dilakukan preprocessing atau feature Engineering)
# Karena data target haruslah original
##
y_valid_model = deserialize_data(path='../data/interim/y_valid.pkl')

In [31]:
def threshold_tuning(trained_model, X_valid, y_valid, metric='f1-score', thresholds=100):
    """
    Melakukan tuning threshold untuk model yang telah dilatih.
    
    Parameters:
    -----------
    trained_model : objek model yang telah dilatih
        Model yang akan digunakan untuk prediksi dan threshold tuning.
    X_valid : array-like
        Data validasi untuk dilakukan prediksi.
    y_valid : array-like
        Data target validasi yang sebenarnya.
    metric : str, opsional, default: 'f1-score'
        Metrics yang akan digunakan untuk evaluasi. Pilihan: 'recall', 'precision', 'f1-score', 'accuracy', dll.
    thresholds : int, default: 100
        Jumlah threshold yang akan dieksplorasi dari 0 hingga 1.
    
    Returns:
    --------
    list
        Daftar nilai threshold yang dieksplorasi.
    """
    # Prediksi probabilitas dari data validasi
    y_pred_proba = trained_model.predict_proba(X_valid)[:, 1]
    
    # Inisialisasi daftar untuk menyimpan hasil metrics
    thresholds_list = np.linspace(0, 1, thresholds)
    metrics_values = []
    
    # Lakukan perulangan untuk setiap nilai threshold
    for threshold in thresholds_list:
        # Lakukan thresholding probabilitas
        y_pred_threshold = (y_pred_proba > threshold).astype(int)
        
        # Hitung metrics menggunakan classification report
        report = classification_report(y_valid, y_pred_threshold, output_dict=True)
        
        # Ambil nilai metrics yang diinginkan
        if metric in report['macro avg']:
            metric_value = report['macro avg'][metric]
        else:
            raise ValueError(f"Metric '{metric}' tidak ditemukan dalam laporan klasifikasi.")
        
        # Simpan hasil metrics
        metrics_values.append({
            'model_name': type(trained_model).__name__,
            'threshold': threshold,
            'metric_value': metric_value,
        })
    
    return metrics_values


Melakukan Threshold Tuning pada model baseline, kNN, DCT

In [33]:
# Melakukan threshold tuning dengan model baseline
baseline_thresholds_data = threshold_tuning(trained_model=baseline_grid_search,
                                    X_valid=X_valid_model,
                                    y_valid=y_valid_model,
                                    metric='f1-score')

c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [34]:
# Melakukan threshold tuning dengan model k-NN
knn_thresholds_data = threshold_tuning(trained_model=knn_grid_search,
                                    X_valid=X_valid_model,
                                    y_valid=y_valid_model,
                                    metric='f1-score')

c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [35]:
# Melakukan threshold tuning dengan model DCT
dct_thresholds_data = threshold_tuning(trained_model=dct_grid_search,
                                    X_valid=X_valid_model,
                                    y_valid=y_valid_model,
                                    metric='f1-score')

c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

Membuat Fungsi untuk Thresholds Terbaik

In [36]:
import json

def select_best_threshold(thresholds_data, save_path):
    """
    Memilih threshold terbaik dan model terbaik berdasarkan metrics yang digunakan.
    
    Parameters:
    -----------
    thresholds_data : list
        Daftar hasil threshold tuning dari semua model.
    save_path : str
        Path untuk menyimpan informasi hasil terbaik.
    
    Returns:
    --------
    float
        Threshold terpilih.
    """
    # Konversi daftar threshold ke dalam DataFrame
    df_thresholds = pd.DataFrame(thresholds_data)
    
    # Urutkan berdasarkan metrics, descending
    df_thresholds_sorted = df_thresholds.sort_values(by='metric_value', ascending=False)
    
    # Ambil threshold terpilih (nilai tertinggi)
    best_threshold = df_thresholds_sorted.iloc[0]['threshold']
    
    # Simpan threshold terpilih ke dalam file JSON
    with open(save_path, 'w') as f:
        json.dump({'best_threshold': best_threshold}, f)
    
    return best_threshold


In [37]:
%pwd

'D:\\Documents\\ai_engineer_pacmann\\ai_engineer_pacmann\\CANDRA_MLPROCESS\\src'

Menyimpan hasil best threshold ke file JSON

In [39]:
# Menyimpan hasil threshold tuning dalam file JSON

# Baseline
baseline_best_threshold = select_best_threshold(thresholds_data = baseline_thresholds_data,
                                       save_path= '../models/baseline_best_threshold.json')

# k-NN
knn_best_threshold = select_best_threshold(thresholds_data = knn_thresholds_data,
                                       save_path= '../models/knn_best_threshold.json')

# dct
dct_best_threshold = select_best_threshold(thresholds_data = dct_thresholds_data,
                                       save_path= '../models/dct_best_threshold.json')

## c. Evaluasion model with Test set (X_test y_test)

In [47]:
# Call the deserialize function to save the dataset into variable

X_test_model = deserialize_data(path='../data/processed/X_test_prep.pkl')

y_test_model = deserialize_data(path='../data/interim/y_test.pkl')

In [49]:
def evaluate_model(model, best_threshold, X_test, y_test):
    """
    Evaluasi model dengan menggunakan threshold terbaik yang telah dipilih.

    Parameters:
    - model: Trained model
    - best_threshold: Selected best threshold
    - X_test: Test data
    - y_test: Test target data

    Returns:
    - classification report
    """
    # Prediksi probabilitas untuk data test
    y_prob = model.predict_proba(X_test)[:, 1]

    # Lakukan thresholding
    y_pred = (y_prob >= best_threshold).astype(int)

    # Print hasil classification report
    print(classification_report(y_test, y_pred))

In [50]:
baseline_best_threshold

np.float64(0.8787878787878789)

In [51]:
# Melakukan evaluasi baseline model
baseline_evaluate = evaluate_model(model= baseline_grid_search,
                                    best_threshold = baseline_best_threshold, 
                                    X_test = X_test_model, 
                                    y_test = y_test_model)

              precision    recall  f1-score   support

           0       0.78      1.00      0.88      2548
           1       0.00      0.00      0.00       711

    accuracy                           0.78      3259
   macro avg       0.39      0.50      0.44      3259
weighted avg       0.61      0.78      0.69      3259



c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [52]:
# Melakukan evaluasi k-NN model
knn_evaluate = evaluate_model(model= knn_grid_search,
                                    best_threshold = knn_best_threshold, 
                                    X_test = X_test_model, 
                                    y_test = y_test_model)

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      2548
           1       0.58      0.63      0.60       711

    accuracy                           0.82      3259
   macro avg       0.74      0.75      0.74      3259
weighted avg       0.83      0.82      0.82      3259



In [53]:
# Melakukan evaluasi DTC model
dtc_evaluate = evaluate_model(model= dct_grid_search,
                                    best_threshold = dct_best_threshold, 
                                    X_test = X_test_model, 
                                    y_test = y_test_model)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2548
           1       0.22      1.00      0.36       711

    accuracy                           0.22      3259
   macro avg       0.11      0.50      0.18      3259
weighted avg       0.05      0.22      0.08      3259



c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f